In [8]:
import numpy as np
import pandas as pd

vocab_dir = '../data/vocabulary_download_v5/'
data_dir='../data/'
rxnorm_relations_table_dir='../RxNorm_relationships_tables/'

import os
directory = rxnorm_relations_table_dir
if not os.path.exists(directory):
    os.makedirs(directory)

In [9]:
c=(pd.read_csv(vocab_dir+'CONCEPT.csv',sep='\t').
  query('vocabulary_id=="RxNorm"').
  loc[:,['concept_id','concept_code','concept_name','concept_class_id']].
  drop_duplicates()
  )
c['concept_id'] = c['concept_id'].astype(int)
cr=(pd.read_csv(vocab_dir+'CONCEPT_RELATIONSHIP.csv',sep='\t').
    loc[:,['concept_id_1','concept_id_2','relationship_id']].
    drop_duplicates()
   )

/tmp/ipykernel_29908/3102070209.py:1: DtypeWarning: Columns (5,6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  c=(pd.read_csv(vocab_dir+'CONCEPT.csv',sep='\t').


In [10]:
cr_verbose = \
(cr.
 set_index('concept_id_1').
 join(
     c.
     set_index('concept_id').
     rename(columns={'concept_name' : 'concept_name_1',
                     'concept_code' : 'concept_code_1',
                     'concept_class_id' : 'concept_class_id_1'})
 ).
 dropna().
 rename_axis('concept_id_1').
 reset_index().
 set_index('concept_id_2').
 join(
     c.
     set_index('concept_id').
     rename(columns={'concept_name' : 'concept_name_2',
                     'concept_code' : 'concept_code_2',
                     'concept_class_id' : 'concept_class_id_2'})
 ).
 dropna().
 rename_axis('concept_id_2').
 reset_index().
 rename(columns={'relationship_id' : 'relationship_id_12'})
)
cr_verbose = cr_verbose.reindex(sorted(cr_verbose.columns),axis=1)
print(cr_verbose.shape)
cr_verbose.head()

(2132680, 9)


,concept_class_id_1,concept_class_id_2,concept_code_1,concept_code_2,concept_id_1,concept_id_2,concept_name_1,concept_name_2,relationship_id_12
0,Clinical Drug,Clinical Drug,252265,252265,903704,903704,tretinoin 0.2 MG/ML Topical Cream,tretinoin 0.2 MG/ML Topical Cream,Mapped from
1,Clinical Drug,Clinical Drug,252265,252265,903704,903704,tretinoin 0.2 MG/ML Topical Cream,tretinoin 0.2 MG/ML Topical Cream,Maps to
2,Clinical Drug,Dose Form,252265,316982,903704,19082224,tretinoin 0.2 MG/ML Topical Cream,Topical Cream,RxNorm has dose form
3,Clinical Drug,Clinical Drug Form,252265,377828,903704,40091090,tretinoin 0.2 MG/ML Topical Cream,tretinoin Topical Cream,RxNorm is a
4,Branded Drug,Branded Drug,284269,284269,903707,903707,tretinoin 0.5 MG/ML Topical Cream [Altinac],tretinoin 0.5 MG/ML Topical Cream [Altinac],Mapped from


In [11]:
cr_verbose.relationship_id_12.value_counts()

relationship_id_12
RxNorm is a             228372
RxNorm inverse is a     228372
Maps to                 208755
Mapped from             208755
Brand name of           119926
Has brand name          119926
RxNorm has dose form    111117
RxNorm dose form of     111117
Has tradename           110295
Tradename of            110295
RxNorm has ing           89816
RxNorm ing of            89816
Constitutes              88135
Consists of              88135
Dose form group of       40541
Has dose form group      40541
Concept replaced by      40291
Concept replaces         40291
Precise ing of           14752
Has precise ing          14752
Has quantified form       7347
Quantified form of        7347
Form of                   3542
Has form                  3542
Contained in              3450
Contains                  3450
Has component                1
Component of                 1
Name: count, dtype: int64

https://www.nlm.nih.gov/research/umls/rxnorm/overview.html

![](../docs/imgs/RxNorm_Drug_Relationships.png)

Each SAB=RXNORM normalized name follows a simple pattern using the ingredient, strength, and dose form. For branded drugs, SAB=RXNORM includes the brand name, and for packs of drugs, SAB=RXNORM includes the quantities of each drug.

- Generic drugs - Ingredient Strength Dose Form

- Branded drugs - Ingredient Strength Dose Form [Brand Name]

- Generic drug packs - {# (Ingredient Strength Dose Form) / # (Ingredient Strength Dose Form)} Pack
- Branded drug packs - {# (Ingredient Strength Dose Form) / # (Ingredient Strength Dose Form)} Pack [Brand Name]

Ingredient: Ingredients generally use the United States Adopted Name (USAN). The Basis of Strength Substance (BoSS) is used in the fully normalized name (e.g., warfarin sodium vs warfarin in 'Warfarin Sodium 1 MG Oral Tablet').

Strength: SAB=RXNORM bases the strength on the active ingredient(s), which may be either the precise (salt or ester) or the base ingredient of the drug. Strengths use the "milli" metric system equivalents (e.g., milligrams, milliliters). Liquids are expressed per milliliter. When a powder can be reconstituted to a range of liquid concentrations, the highest concentration in the range is used for the normalized form.

Dose Form: SAB=RXNORM uses a set of dose forms based upon those included in the Health Level 7 (HL7) data standard.

Brand name: SAB=RXNORM usually represents brand names as seen on the packaging (e.g., Zoloft, Tylenol Cold Relief, and Advair 115/21). In cases where the name on the packaging includes extra information that represents information beyond ingredient or strength, SAB=RXNORM removes that additional information (e.g., Lamisil instead of Lamisil AT and Spiriva instead of Spiriva HandiHaler).

Clinical Drug Comp == IN + Strength

Clinical Drug Form == IN + Dose Form

Clinical Drug Group == IN + Dose Form Group

Clinical Drug == IN + Strength + Dose Form

Branded Drug Comp == IN + Strength + Brand Name

Branded Drug Form == IN + Dose Form + Brand Name

Branded Drug == IN + Strength + Dose Form + Brand Name

Has relationships

In [12]:
rids=cr_verbose[cr_verbose.relationship_id_12.str.contains('[Hh]as')].relationship_id_12.unique()
print(rids)
for rid in rids:
    print(rid)
    tmp = cr_verbose.query('relationship_id_12==@rid')
    print(tmp.shape)
    display(tmp.head())
    display(tmp.concept_class_id_1.value_counts())
    display(tmp.concept_class_id_2.value_counts())
    tmp.to_csv(rxnorm_relations_table_dir+'_'.join(rid.split(' '))+'.csv')

['RxNorm has dose form' 'Has tradename' 'Has brand name' 'RxNorm has ing'
 'Has form' 'Has dose form group' 'Has quantified form' 'Has precise ing'
 'Has component']
RxNorm has dose form
(111117, 9)


,concept_class_id_1,concept_class_id_2,concept_code_1,concept_code_2,concept_id_1,concept_id_2,concept_name_1,concept_name_2,relationship_id_12
2,Clinical Drug,Dose Form,252265,316982,903704,19082224,tretinoin 0.2 MG/ML Topical Cream,Topical Cream,RxNorm has dose form
7,Branded Drug,Dose Form,284269,316982,903707,19082224,tretinoin 0.5 MG/ML Topical Cream [Altinac],Topical Cream,RxNorm has dose form
10,Clinical Drug,Dose Form,313453,346286,903753,19095973,tretinoin 0.001 MG/MG Topical Gel,Topical Gel,RxNorm has dose form
29,Branded Drug,Dose Form,351911,316986,903786,19082228,mequinol 20 MG/ML / tretinoin 0.1 MG/ML Topica...,Topical Solution,RxNorm has dose form
34,Clinical Drug,Dose Form,562793,316986,903791,19082228,mequinol 20 MG/ML / tretinoin 0.1 MG/ML Topica...,Topical Solution,RxNorm has dose form


concept_class_id_1
Clinical Drug          40109
Branded Drug           25460
Branded Drug Form      19046
Clinical Drug Form     17022
Quant Clinical Drug     3990
Quant Branded Drug      3348
Branded Pack            1085
Clinical Pack           1057
Name: count, dtype: int64

concept_class_id_2
Dose Form    111117
Name: count, dtype: int64

Has tradename
(110295, 9)


...

concept_class_id_2
Precise Ingredient    14703
Ingredient               49
Name: count, dtype: int64

Has component
(1, 9)


,concept_class_id_1,concept_class_id_2,concept_code_1,concept_code_2,concept_id_1,concept_id_2,concept_name_1,concept_name_2,relationship_id_12
935326,Clinical Drug,Clinical Drug Form,199442,373831,19021828,40079589,butylscopolamine bromide 10 MG Oral Tablet,scopolamine Oral Tablet,Has component


concept_class_id_1
Clinical Drug    1
Name: count, dtype: int64

concept_class_id_2
Clinical Drug Form    1
Name: count, dtype: int64

Of relationships

In [13]:
rids=cr_verbose[cr_verbose.relationship_id_12.str.contains('[Oo]f')].relationship_id_12.unique()
print(rids)
for rid in rids:
    print(rid)
    tmp = cr_verbose.query('relationship_id_12==@rid')
    print(tmp.shape)
    display(tmp.head())
    display(tmp.concept_class_id_1.value_counts())
    display(tmp.concept_class_id_2.value_counts())
    tmp.to_csv(rxnorm_relations_table_dir+'_'.join(rid.split(' '))+'.csv')

['Tradename of' 'Consists of' 'RxNorm ing of' 'Quantified form of'
 'Brand name of' 'Form of' 'Precise ing of' 'RxNorm dose form of'
 'Dose form group of' 'Component of']
Tradename of
(110295, 9)


,concept_class_id_1,concept_class_id_2,concept_code_1,concept_code_2,concept_id_1,concept_id_2,concept_name_1,concept_name_2,relationship_id_12
6,Branded Drug,Clinical Drug,284269,106303,903707,19008620,tretinoin 0.5 MG/ML Topical Cream [Altinac],tretinoin 0.5 MG/ML Topical Cream,Tradename of
28,Branded Drug,Clinical Drug,351911,562793,903786,903791,mequinol 20 MG/ML / tretinoin 0.1 MG/ML Topica...,mequinol 20 MG/ML / tretinoin 0.1 MG/ML Topica...,Tradename of
42,Branded Drug,Clinical Drug,656882,106303,903808,19008620,tretinoin 0.5 MG/ML Topical Cream [Tretin X],tretinoin 0.5 MG/ML Topical Cream,Tradename of
314,Branded Drug,Clinical Drug,212425,198886,907036,906973,urea 200 MG/ML Topical Cream [Carmol],urea 200 MG/ML Topical Cream,Tradename of
474,Branded Drug,Clinical Drug,795186,308902,908729,19074553,Calcium Carbonate 400 MG / magnesium carbonate...,calcium carbonate 400 MG / magnesium carbonate...,Tradename of


concept_class_id_1
Branded Drug Comp     38230
Branded Drug          25470
Branded Dose Group    23061
Branded Drug Form     19080
Quant Branded Drug     3364
Branded Pack           1090
Name: count, dtype: int64

concept_class_id_2
Clinical Drug Comp     38230
Clinical Drug          25470
Clinical Dose Group    23061
Clinical Drug Form     19080
Quant Clinical Drug     3364
Clinical Pack           1090
Name: count, dtype: int64

Consists of
(88135, 9)


...

concept_class_id_2
Branded Dose Group     22924
Clinical Dose Group    17617
Name: count, dtype: int64

Component of
(1, 9)


,concept_class_id_1,concept_class_id_2,concept_code_1,concept_code_2,concept_id_1,concept_id_2,concept_name_1,concept_name_2,relationship_id_12
1308788,Clinical Drug Form,Clinical Drug,373831,199442,40079589,19021828,scopolamine Oral Tablet,butylscopolamine bromide 10 MG Oral Tablet,Component of


concept_class_id_1
Clinical Drug Form    1
Name: count, dtype: int64

concept_class_id_2
Clinical Drug    1
Name: count, dtype: int64

which tables do I need to convert to all classes to ingredient?

In [14]:
term='Ingredient'
grep='[Hh]as'
rids=cr_verbose[cr_verbose.relationship_id_12.str.contains(grep)].relationship_id_12.unique()
for rid in rids:
    tmp = cr_verbose.query('relationship_id_12==@rid')
    if np.any(tmp.concept_class_id_2.str.contains(term)):
        print(rid)
        display(tmp.head())
        display(tmp.concept_class_id_1.value_counts())
        display(tmp.concept_class_id_2.value_counts())

RxNorm has ing


,concept_class_id_1,concept_class_id_2,concept_code_1,concept_code_2,concept_id_1,concept_id_2,concept_name_1,concept_name_2,relationship_id_12
171,Clinical Drug Comp,Ingredient,316265,6832,904515,904356,methenamine mandelate 1000 MG,methenamine,RxNorm has ing
189,Clinical Drug Comp,Ingredient,336528,6832,904558,904356,methenamine sulfosalicylate 0.02 MG/MG,methenamine,RxNorm has ing
458,Clinical Drug Comp,Ingredient,345897,6972,908643,908523,mineral oil 1 ML/ML,mineral oil,RxNorm has ing
470,Clinical Drug Comp,Ingredient,486186,6972,908703,908523,mineral oil 0.425 MG/MG,mineral oil,RxNorm has ing
529,Clinical Drug Comp,Ingredient,332943,2977,909432,909358,cyclizine 100 MG,cyclizine,RxNorm has ing


concept_class_id_1
Clinical Drug Comp     31622
Clinical Dose Group    30879
Clinical Drug Form     27315
Name: count, dtype: int64

concept_class_id_2
Ingredient            89729
Precise Ingredient       87
Name: count, dtype: int64

Has form


...

concept_class_id_2
Precise Ingredient    3542
Name: count, dtype: int64

Has precise ing


,concept_class_id_1,concept_class_id_2,concept_code_1,concept_code_2,concept_id_1,concept_id_2,concept_name_1,concept_name_2,relationship_id_12
7344,Clinical Drug Comp,Precise Ingredient,637539,159821,1125200,19018324,oxycodone terephthalate 0.38 MG,oxycodone terephthalate,Has precise ing
7350,Clinical Drug Comp,Precise Ingredient,724542,159821,1125232,19018324,oxycodone terephthalate 0.19 MG,oxycodone terephthalate,Has precise ing
8680,Clinical Drug Comp,Precise Ingredient,476533,405970,1141428,19103066,carbinoxamine tannate 0.72 MG/ML,carbinoxamine tannate,Has precise ing
8830,Clinical Drug Comp,Precise Ingredient,2360593,2359,1145529,19055604,chlorhexidine acetate 4.5 MG/ML,chlorhexidine acetate,Has precise ing
9034,Clinical Drug Comp,Precise Ingredient,2371051,71225,1145848,19126898,apomorphine hydrochloride 15 MG,apomorphine hydrochloride,Has precise ing


concept_class_id_1
Brand Name            9433
Clinical Drug Comp    5319
Name: count, dtype: int64

concept_class_id_2
Precise Ingredient    14703
Ingredient               49
Name: count, dtype: int64

what are all the tables that convert to brand name?

In [15]:
term='Brand Name'
grep='[Hh]as'
rids=cr_verbose[cr_verbose.relationship_id_12.str.contains(grep)].relationship_id_12.unique()
for rid in rids:
    tmp = cr_verbose.query('relationship_id_12==@rid')
    if np.any(tmp.concept_class_id_2.str.contains(term)):
        print(rid)
        display(tmp.head())
        display(tmp.concept_class_id_1.value_counts())
        display(tmp.concept_class_id_2.value_counts())

Has brand name


,concept_class_id_1,concept_class_id_2,concept_code_1,concept_code_2,concept_id_1,concept_id_2,concept_name_1,concept_name_2,relationship_id_12
51,Ingredient,Brand Name,28144,216882,903893,19064080,emedastine,Emadine,Has brand name
77,Ingredient,Brand Name,6832,846176,904356,19047656,methenamine,Darpaz,Has brand name
78,Ingredient,Brand Name,6832,220592,904356,19052155,methenamine,UAA,Has brand name
79,Ingredient,Brand Name,6832,220683,904356,19052163,methenamine,Uretron DS,Has brand name
80,Ingredient,Brand Name,6832,220689,904356,19084656,methenamine,Urised,Has brand name


concept_class_id_1
Branded Drug          25280
Branded Drug Comp     24766
Ingredient            23918
Branded Dose Group    22928
Branded Drug Form     19048
Quant Branded Drug     3303
Branded Pack            683
Name: count, dtype: int64

concept_class_id_2
Brand Name    119926
Name: count, dtype: int64

which tables convert the clinical drugs?

In [16]:
term='Clinical'
grep='[Hh]as'
rids=cr_verbose[cr_verbose.relationship_id_12.str.contains(grep)].relationship_id_12.unique()
for rid in rids:
    tmp = cr_verbose.query('relationship_id_12==@rid')
    if np.any(tmp.concept_class_id_1.str.contains(term)):
        print(rid)
        display(tmp.head())
        display(tmp.concept_class_id_1.value_counts())
        display(tmp.concept_class_id_2.value_counts())

RxNorm has dose form


,concept_class_id_1,concept_class_id_2,concept_code_1,concept_code_2,concept_id_1,concept_id_2,concept_name_1,concept_name_2,relationship_id_12
2,Clinical Drug,Dose Form,252265,316982,903704,19082224,tretinoin 0.2 MG/ML Topical Cream,Topical Cream,RxNorm has dose form
7,Branded Drug,Dose Form,284269,316982,903707,19082224,tretinoin 0.5 MG/ML Topical Cream [Altinac],Topical Cream,RxNorm has dose form
10,Clinical Drug,Dose Form,313453,346286,903753,19095973,tretinoin 0.001 MG/MG Topical Gel,Topical Gel,RxNorm has dose form
29,Branded Drug,Dose Form,351911,316986,903786,19082228,mequinol 20 MG/ML / tretinoin 0.1 MG/ML Topica...,Topical Solution,RxNorm has dose form
34,Clinical Drug,Dose Form,562793,316986,903791,19082228,mequinol 20 MG/ML / tretinoin 0.1 MG/ML Topica...,Topical Solution,RxNorm has dose form


concept_class_id_1
Clinical Drug          40109
Branded Drug           25460
Branded Drug Form      19046
Clinical Drug Form     17022
Quant Clinical Drug     3990
Quant Branded Drug      3348
Branded Pack            1085
Clinical Pack           1057
Name: count, dtype: int64

concept_class_id_2
Dose Form    111117
Name: count, dtype: int64

Has tradename


...

concept_class_id_2
Precise Ingredient    14703
Ingredient               49
Name: count, dtype: int64

Has component


,concept_class_id_1,concept_class_id_2,concept_code_1,concept_code_2,concept_id_1,concept_id_2,concept_name_1,concept_name_2,relationship_id_12
935326,Clinical Drug,Clinical Drug Form,199442,373831,19021828,40079589,butylscopolamine bromide 10 MG Oral Tablet,scopolamine Oral Tablet,Has component


concept_class_id_1
Clinical Drug    1
Name: count, dtype: int64

concept_class_id_2
Clinical Drug Form    1
Name: count, dtype: int64

which tables converts the Quants?

In [17]:
term='Quant'
grep='[Hh]as'
rids=cr_verbose[cr_verbose.relationship_id_12.str.contains(grep)].relationship_id_12.unique()
for rid in rids:
    tmp = cr_verbose.query('relationship_id_12==@rid')
    if np.any(tmp.concept_class_id_1.str.contains(term)):
        print(rid)
        display(tmp.head())
        display(tmp.concept_class_id_1.value_counts())
        display(tmp.concept_class_id_2.value_counts())

RxNorm has dose form


,concept_class_id_1,concept_class_id_2,concept_code_1,concept_code_2,concept_id_1,concept_id_2,concept_name_1,concept_name_2,relationship_id_12
2,Clinical Drug,Dose Form,252265,316982,903704,19082224,tretinoin 0.2 MG/ML Topical Cream,Topical Cream,RxNorm has dose form
7,Branded Drug,Dose Form,284269,316982,903707,19082224,tretinoin 0.5 MG/ML Topical Cream [Altinac],Topical Cream,RxNorm has dose form
10,Clinical Drug,Dose Form,313453,346286,903753,19095973,tretinoin 0.001 MG/MG Topical Gel,Topical Gel,RxNorm has dose form
29,Branded Drug,Dose Form,351911,316986,903786,19082228,mequinol 20 MG/ML / tretinoin 0.1 MG/ML Topica...,Topical Solution,RxNorm has dose form
34,Clinical Drug,Dose Form,562793,316986,903791,19082228,mequinol 20 MG/ML / tretinoin 0.1 MG/ML Topica...,Topical Solution,RxNorm has dose form


concept_class_id_1
Clinical Drug          40109
Branded Drug           25460
Branded Drug Form      19046
Clinical Drug Form     17022
Quant Clinical Drug     3990
Quant Branded Drug      3348
Branded Pack            1085
Clinical Pack           1057
Name: count, dtype: int64

concept_class_id_2
Dose Form    111117
Name: count, dtype: int64

Has tradename


...

concept_class_id_2
Branded Drug Comp     38230
Branded Drug          25470
Branded Dose Group    23061
Branded Drug Form     19080
Quant Branded Drug     3364
Branded Pack           1090
Name: count, dtype: int64

Has brand name


,concept_class_id_1,concept_class_id_2,concept_code_1,concept_code_2,concept_id_1,concept_id_2,concept_name_1,concept_name_2,relationship_id_12
51,Ingredient,Brand Name,28144,216882,903893,19064080,emedastine,Emadine,Has brand name
77,Ingredient,Brand Name,6832,846176,904356,19047656,methenamine,Darpaz,Has brand name
78,Ingredient,Brand Name,6832,220592,904356,19052155,methenamine,UAA,Has brand name
79,Ingredient,Brand Name,6832,220683,904356,19052163,methenamine,Uretron DS,Has brand name
80,Ingredient,Brand Name,6832,220689,904356,19084656,methenamine,Urised,Has brand name


concept_class_id_1
Branded Drug          25280
Branded Drug Comp     24766
Ingredient            23918
Branded Dose Group    22928
Branded Drug Form     19048
Quant Branded Drug     3303
Branded Pack            683
Name: count, dtype: int64

concept_class_id_2
Brand Name    119926
Name: count, dtype: int64